In [1]:
import os
import pandas as pd
import sys
import os
from subprocess import call

#%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from IPython.display import display, HTML
import numpy as np

import random
from scipy.stats import ttest_1samp
from sklearn.externals import joblib
from sklearn.linear_model import LassoLarsCV
from matplotlib.lines import Line2D

random.seed(42)
np.random.seed(42)

from scipy.stats import spearmanr, pearsonr

In [4]:
import sys
sys.path.append('../../')

import common_v2.validation_tools

In [5]:
from common_v2.validation_tools import reps

In [7]:
dataset_filetype = 'loaded_full_dataset'
processed_all_rds = pd.read_pickle(f"../../../data/pieces_new/rocklin_all_rds__all_rds_stability.pkl")

In [5]:
path = "../../../data/"
datasets = ['rd1','rd2', 'rd3', 'rd4']

dfs = {lib:pd.read_table(path+lib+"_stability_scores") for lib in datasets}
all_rds = pd.concat(list(dfs.values())) # except ssm2

#all_rds = all_rds.drop_duplicates(subset='name', keep=False) #dropping duplicate records that have the same name. 
# This step is necessary for the correct join

In [6]:
processed_all_rds.shape

(56083, 26)

In [7]:
all_rds.shape

(56183, 18)

In [8]:
all_rds.sequence.unique().shape

(56140,)

In [9]:
train = processed_all_rds[(processed_all_rds.is_train == True) & (processed_all_rds.is_test == False) ]

In [10]:
train.shape

(44942, 26)

In [11]:
train.sequence.unique().shape

(44942,)

In [12]:
train_sequences_with_annotations = np.intersect1d(train.sequence, all_rds.sequence)
train_sequences_with_annotations.shape

(44942,)

In [13]:
train.shape

(44942, 26)

In [14]:
train_names = all_rds.set_index('sequence').loc[train.sequence,['name']]

In [15]:
train_names = train_names[~train_names.index.duplicated(keep='first')]

In [16]:
train = train.set_index('sequence')

In [17]:
train.loc[train_names.index,'fold_topology'] = train_names.values

In [18]:
train.fold_topology.isna().value_counts()

False    44942
Name: fold_topology, dtype: int64

In [19]:
train.fold_topology = train.fold_topology.map(lambda x: x.split("_")[0])

In [20]:
train.fold_topology.value_counts().head(15)

EEHEE                    12942
HEEH                     12376
EHEE                      8847
HHH                       6630
chymo                     1184
tryp                       800
arc                         54
pin1                        51
p53tetS-gabeshortened       31
BBL                         25
GCN4                        20
villin                      12
GCN4-VNVV                   11
hYAP65                      11
PFRD-XC4                     7
Name: fold_topology, dtype: int64

In [21]:
topologies = ["EEHEE", "HEEH", "EHEE", "HHH"]

In [22]:
train = train[train.fold_topology.isin(topologies)]

In [23]:
train.shape

(40795, 26)

In [24]:
models = {}

for top in topologies:
    train_top = train[train.fold_topology == top]
    print(top)
    
    model = LassoLarsCV(
                        fit_intercept = True,
                        normalize = True,
                        n_jobs=-1,
                        max_n_alphas=6000,
                        cv=10
                    )
    
    model.fit(np.asarray(train_top['all_1900'].values.tolist()), train_top['phenotype'].values.astype('float'))
    
    models[top] = model
    print("trained")

EEHEE
trained
HEEH
trained
EHEE
trained
HHH
trained


In [25]:
models

{'EEHEE': LassoLarsCV(copy_X=True, cv=10, eps=2.2204460492503131e-16,
       fit_intercept=True, max_iter=500, max_n_alphas=6000, n_jobs=-1,
       normalize=True, positive=False, precompute='auto', verbose=False),
 'HEEH': LassoLarsCV(copy_X=True, cv=10, eps=2.2204460492503131e-16,
       fit_intercept=True, max_iter=500, max_n_alphas=6000, n_jobs=-1,
       normalize=True, positive=False, precompute='auto', verbose=False),
 'EHEE': LassoLarsCV(copy_X=True, cv=10, eps=2.2204460492503131e-16,
       fit_intercept=True, max_iter=500, max_n_alphas=6000, n_jobs=-1,
       normalize=True, positive=False, precompute='auto', verbose=False),
 'HHH': LassoLarsCV(copy_X=True, cv=10, eps=2.2204460492503131e-16,
       fit_intercept=True, max_iter=500, max_n_alphas=6000, n_jobs=-1,
       normalize=True, positive=False, precompute='auto', verbose=False)}

In [26]:
run_type = 'test'

In [27]:

path = "../../../data/"
datasets = ['rd1','rd2', 'rd3', 'rd4']

dfs = {lib:pd.read_table(path+lib+"_stability_scores") for lib in datasets}
all_rds = pd.concat(list(dfs.values())) # except ssm2

all_rds = all_rds.drop_duplicates(subset='name', keep=False) #dropping duplicate records that have the same name. 
# This step is necessary for the correct join

validate = pd.read_pickle(f"{path}for_rosetta_comparison_rocklin_all_rds_{run_type}_sequences_and_truey.pdpkl")

rock_data = [pd.read_csv(f"{path}rd{i}_relax_scored_filtered_betanov15.sc", sep='\t') for i in [1,2,3,4]]

rock_data = pd.concat([rock_data[x][['sequence', 'description', 'total_score', 'exposed_hydrophobics', 
                                    'buried_np', 
                                    'one_core_each', 
                                    'two_core_each',
                                    'percent_core_SCN',
                                    'buried_minus_exposed', 
                                    'buried_over_exposed', 
                                    'contact_all']] for x in [0,1,2,3]]).set_index('sequence')

validate_meta = all_rds.set_index('sequence').loc[validate.rep].reset_index().copy()

validate_meta.loc[:,'predicted_unirep_fusion_stability'] = np.load(f"{path}rocklin_all_rds__all_rds_stability__all_1900__{run_type}__predictions.npy")

validate_meta.loc[:,'target'] =validate.target

ids_in_common = np.intersect1d(rock_data.description.values,validate_meta.name.dropna().values)

rock_data = rock_data.reset_index().set_index('description')

common_df = validate_meta.set_index('name').loc[ids_in_common]

common_df = common_df.join(rock_data, lsuffix='val', rsuffix='ros_file')

/home/ubuntu/miniconda3/envs/grig_alldatasets_run/lib/python3.6/site-packages/ipykernel/__main__.py:24: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


In [28]:
test = processed_all_rds[(processed_all_rds.is_train == False) & (processed_all_rds.is_test == True) ]

In [29]:
test = test.set_index('sequence')

In [30]:
test.shape

(5570, 25)

In [31]:
common_df.shape

(1432, 29)

In [32]:
common_df.loc[:,"all_1900"] = test.loc[common_df.sequenceval, 'all_1900'].values

In [33]:
common_df = common_df.reset_index()

In [34]:
results = {}

for top in topologies:
    sliced = common_df[common_df.name.map(lambda x: x.split("_")[0]) == top]
    
    results[top] = {
        'UniRep_all_topo':spearmanr(sliced['predicted_unirep_fusion_stability'],sliced['target'])[0],
        'UniRep_single_topo':spearmanr(models[top].predict(np.asarray(sliced['all_1900'].values.tolist())),sliced['target'])[0],
        'buried_NPSA':spearmanr(sliced['buried_np'],sliced['target'])[0],
        'exposed_NPSA':spearmanr(sliced['exposed_hydrophobics'],sliced['target'])[0]
    }

In [41]:
for top in topologies:
    sliced = common_df[common_df.name.map(lambda x: x.split("_")[0]) == top]
    
    results[top]['Rosetta'] = spearmanr(-sliced['total_score'],sliced['target'])[0]

In [45]:
pd.DataFrame(results).loc[['UniRep_single_topo','Rosetta', 'buried_NPSA', 'exposed_NPSA']]

,EEHEE,HEEH,EHEE,HHH
UniRep_single_topo,0.698290,0.434377,0.641517,0.719980
Rosetta,0.597387,0.057320,0.472592,0.596452
buried_NPSA,0.544916,0.044305,0.509138,0.632409
exposed_NPSA,0.549854,0.089763,0.451996,0.060656
